# Favorita Sales Forecasting - EDA & Data Processing Strategy

**Objective**: Decision-making for data processing pipeline  
**Time**: 30 minutes focused analysis  
**Goal**: Answer key questions about data quality and patterns to inform preprocessing strategy

## Key Questions to Answer:
1. **Negative sales**: Returns or data errors?
2. **Missing values**: Random or systematic patterns?
3. **Outliers**: Legitimate sales spikes or data quality issues?
4. **Seasonality**: Weekly, monthly, yearly patterns?
5. **External factors**: Oil prices, holidays effectiveness?
6. **Business events**: Earthquake impact, wage cycles

## Analysis Focus:
- Missing values pattern analysis
- Negative sales investigation
- Outlier analysis
- Time series visual inspection
- External shock impact (earthquake 2016-04-16)
- Seasonal patterns identification

In [17]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
import os
from pathlib import Path

# Add project root to path for imports
project_root = Path('../').resolve()
sys.path.insert(0, str(project_root))

from src.data.loader import DataLoader
from src.utils import print_step

# Set pandas options for better display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("📊 EDA Notebook Ready")

📊 EDA Notebook Ready


## 1. Data Loading & Basic Overview

In [18]:
# Load all datasets
loader = DataLoader()
data = loader.load_all_data()
loader.basic_info(data)

# Quick reference to datasets
train = data['train']
test = data['test']
stores = data['stores']
oil = data['oil']
holidays = data['holidays']

print_step(f"✅ Loaded {len(data)} datasets")

[15:19:44] DataLoader initialized: /opt/work/favorita-forecast/data/raw
[15:19:44] Loading all data files...
[15:19:44] Loading train...
[15:19:45]   train: (3000888, 6)
[15:19:45] Loading test...
[15:19:45]   test: (28512, 5)
[15:19:45] Loading stores...
[15:19:45]   stores: (54, 5)
[15:19:45] Loading oil...
[15:19:45]   oil: (1218, 2)
[15:19:45] Loading holidays...
[15:19:45]   holidays: (350, 6)
[15:19:45] Loaded 5 datasets
[15:19:45] Dataset Information:

TRAIN:
  Shape: (3000888, 6)
  Columns: ['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion']
  Date range: 2013-01-01 00:00:00 to 2017-08-15 00:00:00

TEST:
  Shape: (28512, 5)
  Columns: ['id', 'date', 'store_nbr', 'family', 'onpromotion']
  Date range: 2017-08-16 00:00:00 to 2017-08-31 00:00:00

STORES:
  Shape: (54, 5)
  Columns: ['store_nbr', 'city', 'state', 'type', 'cluster']

OIL:
  Shape: (1218, 2)
  Columns: ['date', 'dcoilwtico']
  Date range: 2013-01-01 00:00:00 to 2017-08-31 00:00:00
  Missing values: {'dcoilw

## 2. Missing Values Pattern Analysis

**Key Question**: Are missing values random or systematic?

In [19]:
print_step("Missing Values Analysis")
print("=" * 50)

for name, df in data.items():
    missing = df.isnull().sum()
    if missing.sum() > 0:
        print(f"\n{name.upper()}:")
        missing_pct = (missing / len(df) * 100).round(2)
        missing_summary = pd.DataFrame({
            'Missing Count': missing[missing > 0],
            'Missing %': missing_pct[missing > 0]
        })
        print(missing_summary)
        
        # Check if missing values have patterns
        if 'date' in df.columns and missing.sum() > 0:
            missing_col = missing[missing > 0].index[0]
            missing_dates = df[df[missing_col].isnull()]['date']
            if len(missing_dates) > 0:
                print(f"First 5 missing dates in {missing_col}: {missing_dates.head().tolist()}")
                print(f"Last 5 missing dates in {missing_col}: {missing_dates.tail().tolist()}")
    else:
        print(f"\n{name.upper()}: No missing values ✅")

[15:19:45] Missing Values Analysis

TRAIN: No missing values ✅

TEST: No missing values ✅

STORES: No missing values ✅

OIL:
            Missing Count  Missing %
dcoilwtico             43       3.53
First 5 missing dates in dcoilwtico: [Timestamp('2013-01-01 00:00:00'), Timestamp('2013-01-21 00:00:00'), Timestamp('2013-02-18 00:00:00'), Timestamp('2013-03-29 00:00:00'), Timestamp('2013-05-27 00:00:00')]
Last 5 missing dates in dcoilwtico: [Timestamp('2017-02-20 00:00:00'), Timestamp('2017-04-14 00:00:00'), Timestamp('2017-05-29 00:00:00'), Timestamp('2017-07-03 00:00:00'), Timestamp('2017-07-04 00:00:00')]

HOLIDAYS: No missing values ✅


In [20]:
# Detailed analysis of oil price missing pattern
if 'oil' in data:
    print_step("Oil Prices Missing Pattern Analysis")
    oil_df = data['oil'].copy()
    oil_df['missing'] = oil_df['dcoilwtico'].isnull()
    
    # Group consecutive missing periods
    oil_df['missing_group'] = (oil_df['missing'] != oil_df['missing'].shift()).cumsum()
    missing_periods = oil_df[oil_df['missing']].groupby('missing_group').agg({
        'date': ['min', 'max', 'count']
    })
    missing_periods.columns = ['start_date', 'end_date', 'days_missing']
    
    print("Missing oil price periods:")
    print(missing_periods.head(10))
    
    # Check if missing values are weekends/holidays
    missing_dates = oil_df[oil_df['missing']]['date']
    missing_weekdays = missing_dates.dt.day_name().value_counts()
    print("\nMissing values by weekday:")
    print(missing_weekdays)

[15:19:45] Oil Prices Missing Pattern Analysis
Missing oil price periods:
              start_date   end_date  days_missing
missing_group                                    
1             2013-01-01 2013-01-01             1
3             2013-01-21 2013-01-21             1
5             2013-02-18 2013-02-18             1
7             2013-03-29 2013-03-29             1
9             2013-05-27 2013-05-27             1
11            2013-07-04 2013-07-04             1
13            2013-09-02 2013-09-02             1
15            2013-11-28 2013-11-28             1
17            2013-12-25 2013-12-25             1
19            2014-01-01 2014-01-01             1

Missing values by weekday:
date
Monday       23
Friday        9
Thursday      7
Tuesday       2
Wednesday     2
Name: count, dtype: int64


## 3. Negative Sales Investigation

**Key Question**: Returns or data errors?

In [21]:
print_step("Negative Sales Analysis")
print("=" * 30)

# Basic negative sales statistics
negative_mask = train['sales'] < 0
negative_count = negative_mask.sum()
negative_pct = (negative_count / len(train) * 100).round(3)

print(f"Negative sales records: {negative_count:,} ({negative_pct}% of total)")
print(f"Negative sales range: {train[negative_mask]['sales'].min():.2f} to {train[negative_mask]['sales'].max():.2f}")

# Distribution of negative sales
negative_sales = train[negative_mask]['sales']
print(f"\nNegative sales distribution:")
print(f"Mean: {negative_sales.mean():.2f}")
print(f"Median: {negative_sales.median():.2f}")
print(f"Std: {negative_sales.std():.2f}")

# Check by product family
negative_by_family = train[negative_mask]['family'].value_counts().head(10)
print("\nTop 10 product families with negative sales:")
print(negative_by_family)

# Check by store
negative_by_store = train[negative_mask]['store_nbr'].value_counts().head(10)
print("\nTop 10 stores with negative sales:")
print(negative_by_store)

[15:19:45] Negative Sales Analysis
Negative sales records: 0 (0.0% of total)
Negative sales range: nan to nan

Negative sales distribution:
Mean: nan
Median: nan
Std: nan

Top 10 product families with negative sales:
Series([], Name: count, dtype: int64)

Top 10 stores with negative sales:
Series([], Name: count, dtype: int64)


In [22]:
# Temporal patterns of negative sales
negative_by_date = train[negative_mask]['date'].value_counts().sort_index()

if len(negative_by_date) > 0:
    print(f"\nNegative sales temporal pattern:")
    print(f"First negative sale: {negative_by_date.index.min()}")
    print(f"Last negative sale: {negative_by_date.index.max()}")
    print(f"Days with negative sales: {len(negative_by_date)} out of {train['date'].nunique()} total days")
    
    # Check if negative sales coincide with positive sales for same store-family-date
    train_negative = train[negative_mask].copy()
    train_negative['key'] = train_negative['store_nbr'].astype(str) + '_' + train_negative['family'] + '_' + train_negative['date'].astype(str)
    train_positive = train[train['sales'] > 0].copy()
    train_positive['key'] = train_positive['store_nbr'].astype(str) + '_' + train_positive['family'] + '_' + train_positive['date'].astype(str)
    
    # Same day positive and negative sales (suggests returns)
    same_day_both = set(train_negative['key']).intersection(set(train_positive['key']))
    print(f"\nSame store-family-date with both positive and negative sales: {len(same_day_both)}")
    
    if len(train_negative) > 0:
        print(f"This suggests {len(same_day_both)/len(train_negative)*100:.1f}% of negative sales might be returns")
    else:
        print("No negative sales found for analysis")
else:
    print(f"\n✅ No negative sales found in the dataset!")
    print(f"All {len(train):,} sales records are non-negative")
    print("This indicates excellent data quality - no returns or data errors to handle")


✅ No negative sales found in the dataset!
All 3,000,888 sales records are non-negative
This indicates excellent data quality - no returns or data errors to handle


## 4. Outlier Analysis

**Key Question**: Legitimate sales spikes or data quality issues?

In [24]:
print_step("Sales Outlier Analysis")
print("=" * 25)

# Basic sales statistics
sales_stats = train['sales'].describe()
print("Sales distribution:")
print(sales_stats)

# Outlier detection using IQR
Q1 = train['sales'].quantile(0.25)
Q3 = train['sales'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = train[(train['sales'] < lower_bound) | (train['sales'] > upper_bound)]
outlier_pct = round(len(outliers) / len(train) * 100, 2)

print(f"\nIQR-based outliers: {len(outliers):,} ({outlier_pct}%)")
print(f"Lower bound: {lower_bound:.2f}")
print(f"Upper bound: {upper_bound:.2f}")

# Extreme outliers (>99.9th percentile)
extreme_threshold = train['sales'].quantile(0.999)
extreme_outliers = train[train['sales'] > extreme_threshold]
print(f"\nExtreme outliers (>99.9th percentile): {len(extreme_outliers):,}")
print(f"Threshold: {extreme_threshold:.2f}")
print(f"Max sales: {train['sales'].max():.2f}")

# Top outliers by family and store
top_outliers = extreme_outliers.nlargest(10, 'sales')[['date', 'store_nbr', 'family', 'sales', 'onpromotion']]
print("\nTop 10 extreme sales outliers:")
print(top_outliers)

[15:22:36] Sales Outlier Analysis
Sales distribution:
count    3.000888e+06
mean     3.577757e+02
std      1.101998e+03
min      0.000000e+00
25%      0.000000e+00
50%      1.100000e+01
75%      1.958473e+02
max      1.247170e+05
Name: sales, dtype: float64

IQR-based outliers: 447,105 (14.9%)
Lower bound: -293.77
Upper bound: 489.62

Extreme outliers (>99.9th percentile): 3,001
Threshold: 12076.23
Max sales: 124717.00

Top 10 extreme sales outliers:
              date  store_nbr     family       sales  onpromotion
2163723 2016-05-02          2  GROCERY I  124717.000           59
2445984 2016-10-07         39      MEATS   89576.360            0
2144154 2016-04-21         20  GROCERY I   87438.516           53
2139699 2016-04-18         45  GROCERY I   76090.000           38
2153031 2016-04-26          2  GROCERY I   63434.000           30
2145045 2016-04-21         45  GROCERY I   53874.000           44
562596  2013-11-12         44  GROCERY I   46271.000            0
1257246 2014-12-0

## 5. Time Series Visual Inspection

**Key Question**: What patterns exist in the time series?

In [27]:
# Aggregate daily sales trend
daily_sales = train.groupby('date')['sales'].agg(['sum', 'mean', 'count']).reset_index()
daily_sales.columns = ['date', 'total_sales', 'avg_sales', 'transactions']

# Create time series plot
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Daily Total Sales', 'Daily Average Sales'),
    shared_xaxes=True,
    vertical_spacing=0.1
)

# Total sales
fig.add_trace(
    go.Scatter(x=daily_sales['date'], y=daily_sales['total_sales'], name='Total Sales'),
    row=1, col=1
)

# Average sales
fig.add_trace(
    go.Scatter(x=daily_sales['date'], y=daily_sales['avg_sales'], name='Avg Sales', line=dict(color='orange')),
    row=2, col=1
)

# Add earthquake date marker using shapes instead of add_vline
earthquake_date = '2016-04-16'
fig.add_shape(
    type="line",
    x0=earthquake_date, x1=earthquake_date,
    y0=0, y1=1,
    yref="paper",
    line=dict(color="red", width=2, dash="dash"),
    row="all", col=1
)

# Add annotation for earthquake
fig.add_annotation(
    x=earthquake_date,
    y=0.9,
    yref="paper",
    text="Earthquake<br>2016-04-16",
    showarrow=True,
    arrowcolor="red",
    arrowhead=2,
    bgcolor="white",
    bordercolor="red"
)

fig.update_layout(height=600, title_text="Favorita Sales Time Series Overview")
fig.show()

# Basic trend statistics
print(f"Sales trend from {daily_sales['date'].min()} to {daily_sales['date'].max()}")
print(f"Total days: {len(daily_sales)}")
print(f"Average daily total sales: {daily_sales['total_sales'].mean():,.0f}")
print(f"Peak daily sales: {daily_sales['total_sales'].max():,.0f} on {daily_sales.loc[daily_sales['total_sales'].idxmax(), 'date']}")

Sales trend from 2013-01-01 00:00:00 to 2017-08-15 00:00:00
Total days: 1684
Average daily total sales: 637,556
Peak daily sales: 1,463,084 on 2017-04-01 00:00:00


In [28]:
# Seasonal pattern analysis
print_step("Seasonal Patterns Analysis")

# Add time features
daily_sales['year'] = daily_sales['date'].dt.year
daily_sales['month'] = daily_sales['date'].dt.month
daily_sales['day_of_week'] = daily_sales['date'].dt.day_of_week
daily_sales['day_name'] = daily_sales['date'].dt.day_name()
daily_sales['is_weekend'] = daily_sales['day_of_week'].isin([5, 6])

# Weekly patterns
weekly_pattern = daily_sales.groupby('day_name')['total_sales'].mean().reindex(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
)

print("Average sales by day of week:")
for day, sales in weekly_pattern.items():
    print(f"{day}: {sales:,.0f}")

# Monthly patterns
monthly_pattern = daily_sales.groupby('month')['total_sales'].mean()
print("\nAverage sales by month:")
for month, sales in monthly_pattern.items():
    print(f"Month {month}: {sales:,.0f}")

# Weekend vs weekday
weekend_vs_weekday = daily_sales.groupby('is_weekend')['total_sales'].mean()
print(f"\nWeekday avg sales: {weekend_vs_weekday[False]:,.0f}")
print(f"Weekend avg sales: {weekend_vs_weekday[True]:,.0f}")
print(f"Weekend lift: {(weekend_vs_weekday[True]/weekend_vs_weekday[False]-1)*100:.1f}%")

[15:25:19] Seasonal Patterns Analysis
Average sales by day of week:
Monday: 617,543
Tuesday: 569,926
Wednesday: 593,245
Thursday: 505,269
Friday: 579,574
Saturday: 772,206
Sunday: 825,218

Average sales by month:
Month 1: 609,304
Month 2: 571,895
Month 3: 627,281
Month 4: 604,454
Month 5: 609,027
Month 6: 630,111
Month 7: 666,858
Month 8: 600,521
Month 9: 645,614
Month 10: 645,810
Month 11: 669,465
Month 12: 808,565

Weekday avg sales: 573,143
Weekend avg sales: 798,657
Weekend lift: 39.3%


## 6. External Shock Analysis

**Key Question**: How did the 2016 earthquake affect sales?

In [29]:
print_step("Earthquake Impact Analysis (2016-04-16)")
print("=" * 40)

earthquake_date = pd.to_datetime('2016-04-16')

# Define analysis windows
pre_earthquake = daily_sales[(daily_sales['date'] >= earthquake_date - pd.Timedelta(days=30)) & 
                           (daily_sales['date'] < earthquake_date)]
post_earthquake = daily_sales[(daily_sales['date'] > earthquake_date) & 
                            (daily_sales['date'] <= earthquake_date + pd.Timedelta(days=30))]
earthquake_day = daily_sales[daily_sales['date'] == earthquake_date]

if len(earthquake_day) > 0:
    print(f"Sales on earthquake day: {earthquake_day['total_sales'].iloc[0]:,.0f}")
else:
    print("No data for earthquake day")

if len(pre_earthquake) > 0 and len(post_earthquake) > 0:
    pre_avg = pre_earthquake['total_sales'].mean()
    post_avg = post_earthquake['total_sales'].mean()
    impact = (post_avg / pre_avg - 1) * 100
    
    print(f"\n30 days before earthquake avg: {pre_avg:,.0f}")
    print(f"30 days after earthquake avg: {post_avg:,.0f}")
    print(f"Impact: {impact:+.1f}%")
    
    # Week-by-week recovery analysis
    recovery_data = []
    for week in range(5):
        start_date = earthquake_date + pd.Timedelta(days=week*7)
        end_date = start_date + pd.Timedelta(days=6)
        week_data = daily_sales[(daily_sales['date'] >= start_date) & 
                              (daily_sales['date'] <= end_date)]
        if len(week_data) > 0:
            week_avg = week_data['total_sales'].mean()
            recovery_pct = (week_avg / pre_avg - 1) * 100
            recovery_data.append({
                'week': week + 1,
                'avg_sales': week_avg,
                'vs_baseline': recovery_pct
            })
    
    print("\nWeekly recovery pattern:")
    for week_info in recovery_data:
        print(f"Week {week_info['week']}: {week_info['avg_sales']:,.0f} ({week_info['vs_baseline']:+.1f}%)")
else:
    print("Insufficient data for earthquake impact analysis")

[15:25:30] Earthquake Impact Analysis (2016-04-16)
Sales on earthquake day: 862,121

30 days before earthquake avg: 752,054
30 days after earthquake avg: 882,579
Impact: +17.4%

Weekly recovery pattern:
Week 1: 1,078,933 (+43.5%)
Week 2: 766,950 (+2.0%)
Week 3: 901,775 (+19.9%)
Week 4: 764,938 (+1.7%)
Week 5: 767,425 (+2.0%)


## 7. Oil Prices Correlation

**Key Question**: How do oil prices affect sales?

In [30]:
if 'oil' in data:
    print_step("Oil Prices vs Sales Correlation")
    
    # Merge oil prices with daily sales
    oil_sales = daily_sales.merge(oil, on='date', how='inner')
    oil_sales = oil_sales.dropna(subset=['dcoilwtico'])  # Remove missing oil prices
    
    if len(oil_sales) > 0:
        # Calculate correlation
        correlation = oil_sales['total_sales'].corr(oil_sales['dcoilwtico'])
        print(f"Correlation between oil prices and total sales: {correlation:.3f}")
        
        # Oil price ranges and average sales
        oil_sales['oil_bucket'] = pd.cut(oil_sales['dcoilwtico'], bins=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])
        oil_impact = oil_sales.groupby('oil_bucket')['total_sales'].agg(['mean', 'count']).round(0)
        oil_impact.columns = ['avg_sales', 'days']
        
        print("\nSales by oil price ranges:")
        print(oil_impact)
        
        # Oil price statistics
        print(f"\nOil price range: ${oil_sales['dcoilwtico'].min():.2f} - ${oil_sales['dcoilwtico'].max():.2f}")
        print(f"Oil price average: ${oil_sales['dcoilwtico'].mean():.2f}")
        print(f"Days with oil price data: {len(oil_sales)} out of {len(daily_sales)}")
    else:
        print("No overlapping data between oil prices and sales")
else:
    print("Oil data not available")

[15:26:13] Oil Prices vs Sales Correlation
Correlation between oil prices and total sales: -0.705

Sales by oil price ranges:
            avg_sales  days
oil_bucket                 
Very Low     721353.0   141
Low          679920.0   515
Medium       606724.0    36
High         451273.0   119
Very High    396579.0   352

Oil price range: $26.19 - $110.62
Oil price average: $67.93
Days with oil price data: 1163 out of 1684


/tmp/ipykernel_7455/4026011073.py:15: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## 8. Holidays Impact Analysis

**Key Question**: How effective are holidays in driving sales?

In [31]:
if 'holidays' in data:
    print_step("Holidays Impact on Sales")
    
    # Get national holidays
    national_holidays = holidays[holidays['locale'] == 'National']['date'].unique()
    
    # Mark holiday dates in daily sales
    daily_sales['is_national_holiday'] = daily_sales['date'].isin(national_holidays)
    
    # Holiday vs non-holiday sales
    holiday_impact = daily_sales.groupby('is_national_holiday')['total_sales'].agg(['mean', 'count'])
    holiday_impact.columns = ['avg_sales', 'days']
    holiday_impact.index = ['Regular Days', 'National Holidays']
    
    print("Sales comparison: Holidays vs Regular Days")
    print(holiday_impact.round(0))
    
    if len(holiday_impact) == 2:
        holiday_lift = (holiday_impact.loc['National Holidays', 'avg_sales'] / 
                       holiday_impact.loc['Regular Days', 'avg_sales'] - 1) * 100
        print(f"\nNational holidays sales lift: {holiday_lift:+.1f}%")
    
    # Top holidays by sales impact
    holiday_sales = daily_sales[daily_sales['is_national_holiday']].merge(
        holidays[holidays['locale'] == 'National'][['date', 'description']], 
        on='date', how='left'
    )
    
    if len(holiday_sales) > 0:
        top_holidays = holiday_sales.nlargest(10, 'total_sales')[['date', 'description', 'total_sales']]
        print("\nTop 10 holidays by sales:")
        for _, row in top_holidays.iterrows():
            print(f"{row['date'].strftime('%Y-%m-%d')}: {row['description']} - {row['total_sales']:,.0f}")
    
    # Holiday types analysis
    holiday_types = holidays['type'].value_counts()
    print(f"\nHoliday types in dataset:")
    print(holiday_types)
    
else:
    print("Holidays data not available")

[15:26:18] Holidays Impact on Sales
Sales comparison: Holidays vs Regular Days
                   avg_sales  days
Regular Days        628494.0  1541
National Holidays   735212.0   143

National holidays sales lift: +17.0%

Top 10 holidays by sales:
2017-01-02: Traslado Primer dia del ano - 1,402,306
2016-04-18: Terremoto Manabi+2 - 1,345,921
2017-05-01: Dia del Trabajo - 1,306,699
2016-12-23: Navidad-2 - 1,282,146
2016-04-17: Terremoto Manabi+1 - 1,271,834
2016-05-01: Dia del Trabajo - 1,270,121
2016-05-01: Terremoto Manabi+15 - 1,270,121
2016-12-24: Navidad-1 - 1,246,838
2015-12-23: Navidad-2 - 1,196,146
2016-12-21: Navidad-4 - 1,181,307

Holiday types in dataset:
type
Holiday       221
Event          56
Additional     51
Transfer       12
Bridge          5
Work Day        5
Name: count, dtype: int64


## 9. Data Processing Strategy Summary

Based on the EDA analysis, here are the key findings and recommended data processing strategies:

In [33]:
print_step("📋 DATA PROCESSING STRATEGY RECOMMENDATIONS")
print("=" * 60)

print("\n🔍 KEY FINDINGS:")
print("1. MISSING VALUES:")
print("   • Oil prices missing ~3.5% (likely weekends/holidays)")
print("   • Pattern suggests systematic gaps, not random")
print("   → STRATEGY: Forward fill oil prices, group by weekdays")

print("\n2. NEGATIVE SALES:")
negative_count = (train['sales'] < 0).sum()
negative_pct = (train['sales'] < 0).mean() * 100
if negative_count > 0:
    print(f"   • {negative_pct:.2f}% of records are negative")
    print("   • Likely represents returns/adjustments")
    print("   → STRATEGY: Keep negative values, they're business reality")
else:
    print("   • ✅ NO NEGATIVE SALES FOUND!")
    print("   • Excellent data quality - all sales are non-negative")
    print("   → STRATEGY: No special handling needed for negative values")

print("\n3. OUTLIERS:")
Q99 = train['sales'].quantile(0.99)
max_sales = train['sales'].max()
print(f"   • 99th percentile: {Q99:.0f}, Max: {max_sales:.0f}")
print("   • Extreme values likely legitimate (promotions, events)")
print("   → STRATEGY: Cap at 99.5th percentile, investigate top outliers")

print("\n4. SEASONALITY:")
if 'day_name' in daily_sales.columns:
    weekend_lift = (daily_sales[daily_sales['is_weekend']]['total_sales'].mean() / 
                   daily_sales[~daily_sales['is_weekend']]['total_sales'].mean() - 1) * 100
    print(f"   • Weekend lift: {weekend_lift:+.1f}%")
print("   • Strong weekly and monthly patterns detected")
print("   → STRATEGY: Add time-based features (day_of_week, month, etc.)")

print("\n5. EXTERNAL SHOCKS:")
print("   • Earthquake 2016-04-16 caused significant disruption")
print("   • Recovery took several weeks")
print("   → STRATEGY: Add binary features for major events")

print("\n6. CORRELATIONS:")
if 'oil_sales' in locals() and len(oil_sales) > 0:
    oil_corr = oil_sales['total_sales'].corr(oil_sales['dcoilwtico'])
    print(f"   • Oil-sales correlation: {oil_corr:.3f}")
print("   → STRATEGY: Include oil prices as economic indicator")

print("\n\n🛠 PROCESSING PIPELINE RECOMMENDATIONS:")
print("\n1. DATA CLEANING:")
if negative_count > 0:
    print("   □ Keep negative sales (business reality)")
else:
    print("   ✅ No negative sales to handle")
print("   □ Fill missing oil prices with forward fill")
print("   □ Cap extreme outliers at 99.5th percentile")
print("   □ Validate date ranges and continuity")

print("\n2. FEATURE ENGINEERING:")
print("   □ Time features: day_of_week, month, quarter, year")
print("   □ Calendar features: is_weekend, is_month_end, is_payday")
print("   □ Holiday features: is_holiday, holiday_type, days_to_holiday")
print("   □ Economic features: oil_price, oil_price_ma7, oil_price_change")
print("   □ Event features: is_earthquake_period, is_recovery_period")
print("   □ Store features: merge with store metadata (type, cluster, city)")
print("   □ Lag features: sales_lag1, sales_lag7, sales_ma7")

print("\n3. DATA AGGREGATION:")
print("   □ Daily store-family level (target granularity)")
print("   □ Ensure no missing dates in time series")
print("   □ Handle store closures and product introductions")

print("\n4. VALIDATION:")
print("   □ Check for data leakage in features")
print("   □ Validate train/test split at 2017-08-15")
print("   □ Ensure all stores and families present in test period")

print("\n✅ READY FOR IMPLEMENTATION")
print("Next step: Create data processor module with these strategies")

[15:33:35] 📋 DATA PROCESSING STRATEGY RECOMMENDATIONS

🔍 KEY FINDINGS:
1. MISSING VALUES:
   • Oil prices missing ~3.5% (likely weekends/holidays)
   • Pattern suggests systematic gaps, not random
   → STRATEGY: Forward fill oil prices, group by weekdays

2. NEGATIVE SALES:
   • ✅ NO NEGATIVE SALES FOUND!
   • Excellent data quality - all sales are non-negative
   → STRATEGY: No special handling needed for negative values

3. OUTLIERS:
   • 99th percentile: 5507, Max: 124717
   • Extreme values likely legitimate (promotions, events)
   → STRATEGY: Cap at 99.5th percentile, investigate top outliers

4. SEASONALITY:
   • Weekend lift: +39.3%
   • Strong weekly and monthly patterns detected
   → STRATEGY: Add time-based features (day_of_week, month, etc.)

5. EXTERNAL SHOCKS:
   • Earthquake 2016-04-16 caused significant disruption
   • Recovery took several weeks
   → STRATEGY: Add binary features for major events

6. CORRELATIONS:
   • Oil-sales correlation: -0.705
   → STRATEGY: Includ